In [ ]:
%%html
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" is="urth-core-import">
<link rel="import" href="urth_components/urth-viz-bar/urth-viz-bar.html" is="urth-core-import">
<link rel="import" href="urth_components/paper-input/paper-input.html" is='urth-core-import' package='PolymerElements/paper-input'>

In [ ]:
import pandas as pd

In [ ]:
fares = pd.read_csv('../demos/tour/fares.csv')
fares.columns = fares.columns.map(lambda x: x.strip())
fares.columns

In [ ]:
%%html
<template id="t" is="dom-bind">
    <urth-core-dataframe id="d" ref="fares" value="{{df}}">
        <urth-core-query-filter disabled="{{!vid}}">vendor_id == '{{vid}}'</urth-core-query-filter>
        <urth-core-query-filter disabled="{{!ptype}}">payment_type == '{{ptype}}'</urth-core-query-filter>
        <urth-core-query-group by="{{gCol}}">
            <urth-core-query-agg op="sum" col="fare_amount"></urth-core-query-agg>
            <urth-core-query-agg op="mean" col="fare_amount"></urth-core-query-agg>
            <urth-core-query-agg op="mean" col="tolls_amount"></urth-core-query-agg>
        </urth-core-query>
    </urth-core-dataframe>
    <paper-input label="By" value="{{gCol}}"></paper-input>
    <paper-input label="Vendor ID" value="{{vid}}"></paper-input>
    <paper-input label="Payment Type" value="{{ptype}}"></paper-input>
    <urth-viz-table datarows="{{df.data}}" columns="{{df.columns}}"></urth-viz-table>
    <urth-viz-bar datarows="{{df.data}}"></urth-viz-bar>
</template>